## XGBSurv Breslow Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/gbdt_pipeline')
from gbdt_pipeline import get_gbdt_pipeline, train_gbdt, train_gbdt_complete

current path /Users/JUSC/Documents/xgbsurv/experiments/eh


## Set Parameters

In [11]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 50
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
model = 'eh'
method = '_gbdt'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        print('split type',type(y))
        if isinstance(y, pd.DataFrame):
            y = y.values[:,0]
        elif isinstance(y, pd.Series):
            try:
                y = y.values[:,0]
            except:
                y = y.values
        elif isinstance(y, np.ndarray):
            try:
                if y.shape[1]>1:
                    y = y[:,0]
            except:
                pass

        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)




## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'eh'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data
    y = data.target

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    y = pd.concat([y,y], axis=1)
    y.columns = ['target1', 'target2']
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y.values[:,1])):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
split type <class 'numpy.ndarray'>
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.frame.DataFrame'>
integration_values.shape[0] 234925
integration_values.shape[0] 47101
Concordance Index 0.6616800920598389
Integrated Brier Score: 0.16823787970209844
{'model': 'eh', 'dataset': 'METABRIC', 'cindex_train': [0.7243445758388155], 'cindex_test': [0.6616800920598389], 'ibs_train': [0.14469901975182992], 'ibs_test': [0.16823787970209844]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.frame.DataFrame'>
integration_values.shape[0] 258566
integration_values.shape[0] 49912
Concordance Index 0.6370430096953725
Integrated Brier Score: 0.17204717118278387
{'model': 'eh', 'dataset': 'METABRIC', 'cindex_train': [0.6991647986342399], 'cindex_test': [0.6370430096953725], 'ibs_train': [0.15390023813663253], 'ibs_test': [0.17204717118278387]}
2
Fitting 5 folds for each of 50 candidates, tot

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,eh,METABRIC,0.724345,0.661680,0.144699,0.168238
1,eh,METABRIC,0.699165,0.637043,0.153900,0.172047
2,eh,METABRIC,0.724863,0.643915,0.145589,0.169233
3,eh,METABRIC,0.686594,0.692411,0.159385,0.157539
4,eh,METABRIC,0.726879,0.657843,0.144049,0.166009
5,eh,FLCHAIN,0.779150,0.796108,0.095639,0.096005
6,eh,FLCHAIN,0.791010,0.799372,0.091889,0.094547
7,eh,FLCHAIN,0.800903,0.780835,0.089917,0.097351
8,eh,FLCHAIN,0.793932,0.778640,0.093702,0.097555
9,eh,FLCHAIN,0.777517,0.795039,0.096190,0.092155


## TCGA Train, Test, Evaluation

In [8]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'eh'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    y = pd.concat([y,y], axis=1)
    y.columns = ['target1', 'target2']
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_gbdt_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
split type <class 'pandas.core.frame.DataFrame'>
0
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.frame.DataFrame'>
integration_values.shape[0] 8540
integration_values.shape[0] 1247
Concordance Index 0.5453038674033149
Integrated Brier Score: 0.22650788422064094
{'model': 'eh', 'dataset': 'BLCA', 'cindex_train': [0.9135661344444854], 'cindex_test': [0.5453038674033149], 'ibs_train': [0.17767186283436523], 'ibs_test': [0.22650788422064094]}
1
Fitting 5 folds for each of 50 candidates, totalling 250 fits
split type <class 'pandas.core.frame.DataFrame'>
integration_values.shape[0] 18259
integration_values.shape[0] 2768
Concordance Index 0.6095030038230476
Integrated Brier Score: 0.22121005741290034
{'model': 'eh', 'dataset': 'BLCA', 'cindex_train': [0.9013912075681692], 'cindex_test': [0.6095030038230476], 'ibs_train': [0.1362340399042315], 'ibs_test': [0.22121005741290034]}
2
Fitting 5 folds for each of 50 candidates, totalli

In [12]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)

In [13]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_test
0,eh,BLCA,0.913566,0.545304,0.177672,0.226508
1,eh,BLCA,0.901391,0.609503,0.136234,0.221210
2,eh,BLCA,0.821206,0.596774,0.187106,0.220653
3,eh,BLCA,0.918913,0.634021,0.134687,0.238788
4,eh,BLCA,0.921053,0.667744,0.153537,0.207585
5,eh,BRCA,0.805337,0.506076,0.155556,0.202736
6,eh,BRCA,0.875311,0.551162,0.156844,0.190763
7,eh,BRCA,0.907091,0.567910,0.130718,0.197140
8,eh,BRCA,0.887353,0.562914,0.114930,0.170867
9,eh,BRCA,0.896816,0.553732,0.099690,0.229136
